In [ ]:
#This function compute interval between two time points
def time_interval(firstdate, seconddate):
    firstday = int(firstdate[8:10])
    firstmonth = int(firstdate[5:7])
    firstyear = int(firstdate[2:4])
    secondday = int(seconddate[8:10])
    secondmonth = int(seconddate[5:7])
    secondyear = int(seconddate[2:4]) 
    first = firstyear * 360 + (firstmonth - 1) * 30 + firstday
    second = secondyear * 360 + (secondmonth - 1) * 30 + secondday
    days = second - first
    return days

In [ ]:
#calculate whodo file_change score
import pandas as pd
import math
address = "results/flink/file_change_num.csv"
numresult_df = pd.read_csv(address)
address = "results/flink/file_change_num.csv"
timeresult_df = pd.read_csv(address)
address = "results/flink/file_change_num.csv"
expresult_df = pd.read_csv(address)
    
address = "features/flink/datedPullFile.csv"
file_df = pd.read_csv(address)
address = "features/flink/datedCommitFileAuthor.csv"
commit_df = pd.read_csv(address)
for i in range(numresult_df.shape[0]):
    number = numresult_df.loc[i, 'number']
    print(i)
    print(number)
    myfile_df = file_df[file_df.pull_number == number]
    if myfile_df.shape[0] == 0:
        continue
    date = myfile_df.iloc[0, 2]
    for j in range(myfile_df.shape[0]):
        mycommit_df = commit_df[commit_df.date < date]
        path = myfile_df.iloc[j, 1]
        newcommit_df = mycommit_df[mycommit_df.file_path==path]
        num_df = newcommit_df.groupby(['author_login'])['oid'].count()
        time_df = newcommit_df.groupby(['author_login'])['date'].max()
        for k in range(num_df.shape[0]):
            committer = num_df.index[k]
            num = num_df.iloc[k]
            max_value = time_df.iloc[k]
            numresult_df.loc[i, committer] += num
            lasttime = time_interval(max_value, date)
            if lasttime !=0:
                timeresult_df.loc[i, committer] += num / lasttime
            else:
                timeresult_df.loc[i, committer] += num
            expresult_df.loc[i, committer] += num / math.exp(lasttime/7)

numresult_df.to_csv("results/flink/file_change_num.csv", index = False)
timeresult_df.to_csv("results/flink/file_change_time.csv", index = False)
expresult_df.to_csv("results/flink/file_change_exp.csv", index = False)
timeresult_df

In [ ]:
#calculate whodo pach_change score
import pandas as pd
import math
address = "results/beam/pack_change_num.csv"
timeresult_df = pd.read_csv(address)
address = "features/beam/datedPullFile.csv"
file_df = pd.read_csv(address)
address = "features/beam/datedCommitFileAuthor.csv"
commit_df = pd.read_csv(address)
def function_r(x):
    A = x.split("/")
    del A[-1]
    return('/'.join(A))
file_df['file_path'] = file_df['file_path'].apply(function_r)
for i in range(numresult_df.shape[0]):
    number = numresult_df.loc[i, 'number']
    print(i)
    print(number)
    myfile_df = file_df[file_df.pull_number == number]
    myfile_df = myfile_df.drop_duplicates(subset=['file_path'], keep="first")
    if myfile_df.shape[0] == 0:
        continue
    date = myfile_df.iloc[0, 2]
    for j in range(myfile_df.shape[0]):
        mycommit_df = commit_df[commit_df.date < date]
        path = myfile_df.iloc[j, 1]
        newcommit_df = mycommit_df[mycommit_df.file_path.str.startswith(path)]
        newcommit_df = newcommit_df.drop_duplicates(subset=['oid', 'author_login'], keep="last")
        num_df = newcommit_df.groupby(['author_login'])['oid'].count()
        time_df = newcommit_df.groupby(['author_login'])['date'].max()
        for k in range(num_df.shape[0]):
            committer = num_df.index[k]
            num = num_df.iloc[k]
            max_value = time_df.iloc[k]
            lasttime = time_interval(max_value, date)
            if lasttime !=0:
                timeresult_df.loc[i, committer] += num / lasttime
            else:
                timeresult_df.loc[i, committer] += num
timeresult_df.to_csv("results/beam/pack_change_time.csv", index = False)

In [ ]:
#calculate whodo file_review score
import pandas as pd
import math
address = "results/zookeeper/file_review_num.csv"
numresult_df = pd.read_csv(address)
address = "results/zookeeper/file_review_num.csv"
timeresult_df = pd.read_csv(address)
address = "results/zookeeper/file_review_num.csv"
expresult_df = pd.read_csv(address)
address = "features/spark/datedPullFile.csv"
file_df = pd.read_csv(address)
address = "features/spark/datedReviewFile.csv"
review_df = pd.read_csv(address)

for i in range(numresult_df.shape[0]):
    number = numresult_df.loc[i, 'number']
    print(i)
    print(number)
    myfile_df = file_df[file_df.pull_number == number]
    if myfile_df.shape[0] == 0:
        continue
    date = myfile_df.iloc[0, 2]
    for j in range(myfile_df.shape[0]):
        myreview_df = review_df[review_df.date < date]
        path = myfile_df.iloc[j, 1]
        newreview_df = myreview_df[myreview_df.file_path == path]
        newreview_df = newreview_df.drop_duplicates(subset=['pull_number', 'reviewer_login'], keep="last")
        num_df = newreview_df.groupby(['reviewer_login'])['pull_number'].count()
        time_df = newreview_df.groupby(['reviewer_login'])['date'].max()
        for k in range(num_df.shape[0]):
            reviewer = num_df.index[k]
            num = num_df.iloc[k]
            max_value = time_df.iloc[k]
            numresult_df.loc[i, reviewer] += num
            lasttime = time_interval(max_value, date)
            if lasttime !=0:
                timeresult_df.loc[i, reviewer] += num / lasttime
            else:
                timeresult_df.loc[i, reviewer] += num
            expresult_df.loc[i, reviewer] += num / math.exp(lasttime/7)

numresult_df.to_csv("results/spark/file_review_num.csv", index = False)
timeresult_df.to_csv("results/spark/file_review_time.csv", index = False)
expresult_df.to_csv("results/spark/file_review_exp.csv", index = False)
timeresult_df

In [ ]:
#calculate whodo pack_review score
import pandas as pd
import math
address = "results/spark/pack_review_num.csv"
timeresult_df = pd.read_csv(address)
address = "features/spark/datedPullFile.csv"
file_df = pd.read_csv(address)
address = "features/spark/datedReviewFile.csv"
review_df = pd.read_csv(address)
def function_r(x):
    A = x.split("/")
    del A[-1]
    return('/'.join(A))
file_df['file_path'] = file_df['file_path'].apply(function_r)
for i in range(numresult_df.shape[0]):
    number = numresult_df.loc[i, 'number']
    print(i)
    print(number)
    myfile_df = file_df[file_df.pull_number == number]
    myfile_df = myfile_df.drop_duplicates(subset=['file_path'], keep="first")
    if myfile_df.shape[0] == 0:
        continue
    date = myfile_df.iloc[0, 2]
    for j in range(myfile_df.shape[0]):
        myreview_df = review_df[review_df.date < date]
        path = myfile_df.iloc[j, 1]
        newreview_df = myreview_df[myreview_df.file_path.str.startswith(path)]
        newreview_df = newreview_df.drop_duplicates(subset=['pull_number', 'reviewer_login'], keep="last")
        num_df = newreview_df.groupby(['reviewer_login'])['pull_number'].count()
        time_df = newreview_df.groupby(['reviewer_login'])['date'].max()
        for k in range(num_df.shape[0]):
            reviewer = num_df.index[k]
            num = num_df.iloc[k]
            max_value = time_df.iloc[k]
            lasttime = time_interval(max_value, date)
            if lasttime !=0:
                timeresult_df.loc[i, reviewer] += num / lasttime
            else:
                timeresult_df.loc[i, reviewer] += num
timeresult_df.to_csv("results/spark/pack_review_time.csv", index = False)

In [ ]:
#calculate final whodo_scores
import pandas as pd
address = "results/spark/file_change_time.csv"
filechange_df = pd.read_csv(address)
filechange_df = filechange_df.iloc[:, 1:]
address = "results/spark/file_review_time.csv"
filereview_df = pd.read_csv(address)
filereview_df = filereview_df.iloc[:, 1:]
address = "results/spark/pack_change_time.csv"
packchange_df = pd.read_csv(address)
packchange_df = packchange_df.iloc[:, 1:]
address = "results/spark/pack_review_time.csv"
packreview_df = pd.read_csv(address)
packreview_df = packreview_df.iloc[:, 1:]
ans_df = filechange_df + filereview_df + packchange_df + packreview_df
ans_df.to_csv("results/spark/whodo_scores.csv", index = False)